In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')
data.head

<bound method NDFrame.head of             Date     Open     High      Low    Close    Volume
0     2016-01-04  1099.59  1110.75  1093.85  1109.64  66545000
1     2016-01-05  1117.47  1130.57  1112.75  1125.85  41227200
2     2016-01-06  1116.63  1138.68  1116.10  1133.04  42316800
3     2016-01-07  1122.00  1149.76  1117.56  1143.37  64899600
4     2016-01-08  1145.64  1149.96  1120.93  1122.88  52138900
...          ...      ...      ...      ...      ...       ...
1650  2022-07-25  2412.25  2429.44  2387.41  2402.95  29106300
1651  2022-07-26  2289.29  2307.08  2255.66  2280.27  48503400
1652  2022-07-27  2309.14  2404.25  2296.82  2395.54  43790900
1653  2022-07-28  2385.58  2443.26  2377.34  2434.89  25807000
1654  2022-07-29  2410.86  2479.24  2406.35  2474.93  32907600

[1655 rows x 6 columns]>

In [3]:
training_set = data.iloc[:, 1:2].values

In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
training_set_scaled.shape

(1655, 1)

In [5]:
X_train = []
y_train = []
for i in range(60, 1655):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [8]:
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=30,batch_size=32)

Epoch 1/30
50/50 [==============================] - 11s 86ms/step - loss: 0.0243
Epoch 2/30
50/50 [==============================] - 5s 100ms/step - loss: 0.0048
Epoch 3/30
50/50 [==============================] - 5s 107ms/step - loss: 0.0046
Epoch 4/30
50/50 [==============================] - 5s 99ms/step - loss: 0.0037
Epoch 5/30
50/50 [==============================] - 5s 106ms/step - loss: 0.0041
Epoch 6/30
50/50 [==============================] - 6s 123ms/step - loss: 0.0033
Epoch 7/30
50/50 [==============================] - 8s 151ms/step - loss: 0.0035
Epoch 8/30
50/50 [==============================] - 6s 115ms/step - loss: 0.0028
Epoch 9/30
50/50 [==============================] - 6s 119ms/step - loss: 0.0033
Epoch 10/30
50/50 [==============================] - 6s 128ms/step - loss: 0.0033
Epoch 11/30
50/50 [==============================] - 5s 100ms/step - loss: 0.0028
Epoch 12/30
50/50 [==============================] - 5s 101ms/step - loss: 0.0029
Epoch 13/30
50/50 [=======

In [9]:
dataset_test = pd.read_csv('test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
real_stock_price.shape

(35, 1)

In [10]:
dataset_total = pd.concat((data['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, len(inputs)):
        X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

C:\Users\Ram Selvaraj\AppData\Local\Temp\ipykernel_19548\3587678286.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values


2/2 [==============================] - 2s 18ms/step


In [11]:
dataset_test['close'] = predicted_stock_price

In [12]:
dataset_test = dataset_test.drop(['Open'] , axis = 1)
dataset_test = dataset_test.drop(['High'] , axis = 1)
dataset_test = dataset_test.drop(['Low'] , axis = 1)
dataset_test = dataset_test.drop(['Volume'] , axis = 1)

In [13]:
dataset_test = dataset_test.set_index('Date') 

In [27]:
dataset_test.head()

,close
Date,
2022-08-01,2418.952881
2022-08-02,2415.322510
2022-08-03,2413.021973
2022-08-04,2412.967285
2022-08-05,2415.537109


### CHECKING ERROR IN LSTM MODEL

In [31]:
lstmTest = pd.read_csv('predicted_submission.csv')


In [32]:
print("Root Mean Squared Errror:",np.sqrt(mean_squared_error(lstmTest['Close'],dataset_test['close'])))

Root Mean Squared Errror: 73.09745046844495


# PREDICTING USING LINEAR REGRESSION

In [15]:
df = pd.read_csv('train.csv')
df

,Date,Open,High,Low,Close,Volume
0,2016-01-04,1099.59,1110.75,1093.85,1109.64,66545000
1,2016-01-05,1117.47,1130.57,1112.75,1125.85,41227200
2,2016-01-06,1116.63,1138.68,1116.10,1133.04,42316800
3,2016-01-07,1122.00,1149.76,1117.56,1143.37,64899600
4,2016-01-08,1145.64,1149.96,1120.93,1122.88,52138900
...,...,...,...,...,...,...
1650,2022-07-25,2412.25,2429.44,2387.41,2402.95,29106300
1651,2022-07-26,2289.29,2307.08,2255.66,2280.27,48503400
1652,2022-07-27,2309.14,2404.25,2296.82,2395.54,43790900
1653,2022-07-28,2385.58,2443.26,2377.34,2434.89,25807000


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['Open','High','Low','Volume']], df[['Close']], test_size=.2)

In [17]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
print("Root Mean Squared Errror:",np.sqrt(mean_squared_error(y_test,y_pred)))
df_pred = pd.DataFrame(y_test.values, columns=['Actual'], index=y_test.index)
df_pred['Predicted'] = y_pred
df_pred

Root Mean Squared Errror: 7.733657958812796


,Actual,Predicted
736,1597.02,1608.329883
1361,2608.95,2607.422740
1382,2636.11,2635.881087
872,1794.13,1801.415007
805,1684.79,1682.781328
...,...,...
915,1824.12,1823.216280
1160,2153.99,2141.117990
92,1151.08,1149.440360
1433,2835.85,2837.042714


### RMSE ERROR OF 7.733 for LR, Clearly much less than the 73 in lstm

## RUNNING FOR TEST CSV-

In [22]:
dftest = pd.read_csv('test.csv')
dftest.head()

,Date,Open,High,Low,Volume
0,2022-08-01,2455.71,2500.26,2447.87,24327600
1,2022-08-02,2466.68,2494.73,2447.99,19076300
2,2022-08-03,2473.29,2507.59,2449.39,27459700
3,2022-08-04,2475.98,2487.75,2419.88,23699600
4,2022-08-05,2414.66,2444.17,2409.01,18912400


In [25]:
y_testpred = model.predict(dftest[['Open','High','Low','Volume']])
dftestnew = dftest
dftestnew['Close'] = y_testpred
dftestnew[['Date','Close']].head()
#dftestnew.reset_index(inplace = True)
dftestnew[['Date','Close']].to_csv('LRPred.csv', index=False)
dftestnew[['Date','Close']].head()

,Date,Close
0,2022-08-01,2485.103225
1,2022-08-02,2473.488938
2,2022-08-03,2480.619906
3,2022-08-04,2437.447200
4,2022-08-05,2433.920886


In [26]:
pred = dftestnew
actual = pd.read_csv('predicted_submission.csv')
print("Root Mean Squared Errror:",np.sqrt(mean_squared_error(actual['Close'],pred['Close'])))

Root Mean Squared Errror: 0.6753815216705865


### CLEARLY VERY SIMILAR TO PREDICTED SUBMISSION CSV